<a href="https://colab.research.google.com/github/Praharshita1275/Criminal_mind_analysis/blob/main/Criminal_mind_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# path = "https://drive.google.com/drive/folders/1KPv1lCH1Pol7HikFF7Qi8S5G3JbTACd3?usp=drive_link


#viraj


## llm1


In [3]:
import pandas as pd
import numpy as np


In [4]:
#load dataset

df = pd.read_csv("/content/Crime_Data_from_2020_to_Present.csv")

df.head()
df.info()
df.isna().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 326977 entries, 0 to 326976
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   DATE OCC      326977 non-null  object 
 1   TIME OCC      326977 non-null  int64  
 2   AREA NAME     326977 non-null  object 
 3   Vict Age      326977 non-null  int64  
 4   Vict Sex      326977 non-null  object 
 5   Vict Descent  326977 non-null  object 
 6   Premis Desc   326977 non-null  object 
 7   Weapon Desc   326977 non-null  object 
 8   Status Desc   326977 non-null  object 
 9   LOCATION      326977 non-null  object 
 10  LAT           326977 non-null  float64
 11  LON           326977 non-null  float64
 12  Crm Cd Desc   326977 non-null  object 
dtypes: float64(2), int64(2), object(9)
memory usage: 32.4+ MB


,0
DATE OCC,0
TIME OCC,0
AREA NAME,0
Vict Age,0
Vict Sex,0
Vict Descent,0
Premis Desc,0
Weapon Desc,0
Status Desc,0
LOCATION,0


In [5]:

#standardise column names

df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
    .str.replace(r"[^a-z0-9_]", "", regex=True)
)

df.columns


Index(['date_occ', 'time_occ', 'area_name', 'vict_age', 'vict_sex',
       'vict_descent', 'premis_desc', 'weapon_desc', 'status_desc', 'location',
       'lat', 'lon', 'crm_cd_desc'],
      dtype='object')

In [6]:
#HANDLE MISSING & INVALID VALUES
text_cols = [
    "area_name", "premis_desc", "weapon_desc",
    "status_desc", "crm_cd_desc", "location"
]

for col in text_cols:
    df[col] = df[col].fillna("UNKNOWN")

df["vict_age"] = df["vict_age"].replace(0, np.nan)
df["vict_age"] = df["vict_age"].fillna("UNKNOWN")



In [7]:
# BASIC TEXT CLEANING (LIGHT)
def clean_text(col):
    return (
        col.astype(str)
        .str.lower()
        .str.replace(r"\s+", " ", regex=True)
        .str.strip()
    )

for col in text_cols:
    df[col] = clean_text(df[col])


In [8]:
# FIX DATE & TIME
df["date_occ"] = pd.to_datetime(df["date_occ"], errors="coerce")

def time_to_hour(x):
    try:
        x = int(x)
        return x // 100
    except:
        return "UNKNOWN"

df["time_hour"] = df["time_occ"].apply(time_to_hour)


/tmp/ipython-input-1132658626.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["date_occ"] = pd.to_datetime(df["date_occ"], errors="coerce")


In [9]:
# SELECT ONLY USEFUL COLUMNS
keep_cols = [
    "date_occ", "time_hour", "area_name",
    "vict_age", "vict_sex",
    "premis_desc", "weapon_desc",
    "status_desc", "crm_cd_desc",
    "location"
]

df = df[keep_cols]



In [10]:
# CREATE THE MOST IMPORTANT COLUMN (crime_text)
""" important step """
def create_crime_text(row):
    return (
        f"On {row['date_occ'].date()} at {row['time_hour']} hours, "
        f"in {row['area_name']} area, a {row['vict_age']}-year-old "
        f"{row['vict_sex']} was involved in {row['crm_cd_desc']} "
        f"at {row['premis_desc']}. "
        f"Weapon used: {row['weapon_desc']}. "
        f"Case status: {row['status_desc']}."
    )

df["crime_text"] = df.apply(create_crime_text, axis=1)


In [11]:
df["crime_text"].head(3)


,crime_text
0,"On 2020-05-10 at 22 hours, in central area, a ..."
1,"On 2020-12-02 at 22 hours, in 77th street area..."
2,"On 2020-05-01 at 23 hours, in 77th street area..."


In [12]:
#ADD SIMPLE RULE-BASED MOTIVATION LABEL
def infer_motivation(crime):
    crime = crime.lower()
    if "robbery" in crime or "theft" in crime:
        return "financial"
    elif "intimate partner" in crime or "rape" in crime:
        return "emotional"
    elif "assault" in crime or "weapon" in crime:
        return "power"
    else:
        return "unknown"

df["initial_motivation"] = df["crm_cd_desc"].apply(infer_motivation)


In [13]:

print(df["crime_text"][1])

On 2020-12-02 at 22 hours, in 77th street area, a 21.0-year-old M was involved in robbery at street. Weapon used: verbal threat. Case status: invest cont.


In [16]:
# Save processed data as CSV
df.to_csv("processed_crime_data.csv", index=False)

# -------------------------------
# Prepare JSON data for LLM usage
# -------------------------------

llm_data = []

for _, row in df.iterrows():
    llm_data.append({
        "text": row["crime_text"],
        "area": row["area_name"],
        "crime_type": row["crm_cd_desc"],
        "weapon": row["weapon_desc"],
        "motivation_hint": row["initial_motivation"]
    })

# Save LLM-ready JSON file
import json

with open("crime_data_llm_ready.json", "w", encoding="utf-8") as f:
    json.dump(llm_data, f, indent=2, ensure_ascii=False)


In [15]:
"""

# ============================================
# STEP 11 — LOAD LLM-READY JSON
# ============================================

import json
import torch
from transformers import pipeline
from sentence_transformers import SentenceTransformer, util

# Load preprocessed data
with open("crime_data_llm_ready.json", "r") as f:
    llm_data = json.load(f)

# Sanity check
print("Sample record:")
print(llm_data[0])
print("-" * 50)


# ============================================
# STEP 12 — ZERO-SHOT CRIME CLASSIFICATION (LLM-1)
# ============================================

classifier = pipeline(
    "zero-shot-classification",
    model="facebook/bart-large-mnli"
)

candidate_labels = [
    "violent crime",
    "financial crime",
    "sexual crime",
    "domestic violence",
    "property crime",
    "non criminal"
]

# Test on one sample first
sample_text = llm_data[0]["text"]
sample_result = classifier(sample_text, candidate_labels)

print("Zero-shot test result:")
print(sample_result)
print("-" * 50)


# ============================================
# STEP 13 — ATTACH LLM-1 OUTPUT TO DATA
# ============================================

for item in llm_data:
    res = classifier(item["text"], candidate_labels)
    item["llm1_predicted_category"] = res["labels"][0]
    item["llm1_score"] = res["scores"][0]


# ============================================
# STEP 14 — SEMANTIC SIMILARITY (PATTERN SCORING)
# ============================================

embedder = SentenceTransformer("all-MiniLM-L6-v2")

# Crime motivation prototypes
prototypes = {
    "financial": [
        "robbery for money",
        "stealing cash",
        "financial gain from theft"
    ],
    "emotional": [
        "domestic dispute",
        "relationship conflict",
        "crime due to emotional distress"
    ],
    "power": [
        "assault using weapon",
        "threatening with force",
        "crime to show dominance"
    ]
}

# Encode prototype embeddings
proto_embeds = {
    label: embedder.encode(texts, convert_to_tensor=True)
    for label, texts in prototypes.items()
}

# Compute semantic similarity
for item in llm_data:
    text_emb = embedder.encode(item["text"], convert_to_tensor=True)

    best_label = None
    best_score = 0.0

    for label, proto_emb in proto_embeds.items():
        score = torch.max(util.cos_sim(text_emb, proto_emb)).item()
        if score > best_score:
            best_label = label
            best_score = score

    item["semantic_pattern"] = best_label
    item["semantic_score"] = best_score


# ============================================
# SAVE UPDATED DATASET
# ============================================

with open("crime_data_llm1_enriched.json", "w") as f:
    json.dump(llm_data, f, indent=2)

print("LLM-1 enrichment complete.")
print("Saved as crime_data_llm1_enriched.json")


"""


'\n\n# ============================================\n# STEP 11 — LOAD LLM-READY JSON\n# ============================================\n\nimport json\nimport torch\nfrom transformers import pipeline\nfrom sentence_transformers import SentenceTransformer, util\n\n# Load preprocessed data\nwith open("crime_data_llm_ready.json", "r") as f:\n    llm_data = json.load(f)\n\n# Sanity check\nprint("Sample record:")\nprint(llm_data[0])\nprint("-" * 50)\n\n\n# ============================================\n# STEP 12 — ZERO-SHOT CRIME CLASSIFICATION (LLM-1)\n# ============================================\n\nclassifier = pipeline(\n    "zero-shot-classification",\n    model="facebook/bart-large-mnli"\n)\n\ncandidate_labels = [\n    "violent crime",\n    "financial crime",\n    "sexual crime",\n    "domestic violence",\n    "property crime",\n    "non criminal"\n]\n\n# Test on one sample first\nsample_text = llm_data[0]["text"]\nsample_result = classifier(sample_text, candidate_labels)\n\nprint("Zero-

## llm2


In [17]:
import pandas as pd
import json

# ===============================
# STEP 1: Load Murder Motives Data
# ===============================
df = pd.read_csv("/content/Murder Motives.csv")

# ===============================
# STEP 2: Clean Column Names
# ===============================
df.columns = (
    df.columns.str.lower()
              .str.replace(" ", "_")
              .str.replace("/", "_")
)

# ===============================
# STEP 3: Verify Column Names
# ===============================
# Check if the necessary column exists
required_columns = [
    "gain",
    "property_dispute",
    "personal_vendetta_or_enemity",
    "love_affairs_sexual_relations",
    "dowry",
    "communalism",
    "casteism",
    "political_reasons",
    "terrorists_extremists",
    "other_causes"
]

# Print the actual column names in the DataFrame to debug
print("Actual column names in the DataFrame:", df.columns)

# Check if the required columns exist in the DataFrame
missing_cols = [col for col in required_columns if col not in df.columns]
if missing_cols:
    print(f"Warning: The following columns are missing in the dataset: {missing_cols}")
else:
    print("All required columns are present.")

# ===============================
# STEP 4: Remove Aggregate Rows
# ===============================
df = df[~df["state"].str.contains("TOTAL", na=False)]

# ===============================
# STEP 5: Identify Motivation Columns (LLM-2)
# ===============================
# Ensure the column names match with the cleaned names
motivation_cols = [
    "gain",
    "property_dispute",
    "personal_vendetta_or_enemity",
    "love_affairs_sexual_relations",  # This should match the column name exactly
    "dowry",
    "communalism",
    "casteism",
    "political_reasons",
    "terrorists_extremists",
    "other_causes"
]

# ===============================
# STEP 6: Convert Each Row to Motivation Profile
# ===============================
llm2_data = []

for _, row in df.iterrows():
    # Build motivation profile for each row
    motivation_profile = {
        col: int(row[col]) if not pd.isna(row[col]) else 0
        for col in motivation_cols if col in df.columns  # Ensure the column exists
    }

    llm2_data.append({
        "state": row["state"],
        "year": row["year"],
        "motivation_distribution": motivation_profile
    })

# ===============================
# STEP 7: Save LLM-2 Dataset
# ===============================
with open("llm2_motivation_dataset.json", "w") as f:
    json.dump(llm2_data, f, indent=2)

print("✅ LLM-2 Motivation Dataset Created")


Actual column names in the DataFrame: Index(['state', 'year', 'gain', 'property_dispute',
       'personal_vendetta_or_enemity', 'love_affairs__sexual_relations',
       'dowry', 'lunacy', 'witchcraft', 'communalism', 'casteism',
       'class_conflict', 'political_reasons', 'terrorists__extremists',
       'other_causes', 'total'],
      dtype='object')
✅ LLM-2 Motivation Dataset Created


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

## llm 3

In [ ]:
import pandas as pd
import json

# ===============================
# STEP 1: Load Chicago Crimes Data
# ===============================
df = pd.read_csv("/content/Chicago_Crimes_2022.csv")

# ===============================
# STEP 2: Standardize Column Names
# ===============================
df.columns = (
    df.columns.str.lower()
              .str.replace(" ", "_")
              .str.replace(r"[^a-z0-9_]", "", regex=True)
)

# ===============================
# STEP 3: Select Context-Relevant Columns (LLM-3)
# ===============================
llm3_cols = [
    "date",
    "primary_type",
    "description",
    "location_description",
    "arrest",
    "domestic",
    "beat",
    "district",
    "ward",
    "community_area",
    "year",
    "latitude",
    "longitude"
]

df_llm3 = df[llm3_cols].fillna("UNKNOWN")

# ===============================
# STEP 4: Create Context Narrative (Text for LLM)
# ===============================
def build_context_text(row):
    return (
        f"In {row['year']}, a {row['primary_type']} incident occurred at a "
        f"{row['location_description']} location. "
        f"Domestic case: {row['domestic']}. "
        f"Arrest made: {row['arrest']}. "
        f"District {row['district']}, Beat {row['beat']}."
    )

df_llm3["context_text"] = df_llm3.apply(build_context_text, axis=1)

# ===============================
# STEP 5: Convert to LLM-3 JSON Format
# ===============================
llm3_data = []

for _, row in df_llm3.iterrows():
    llm3_data.append({
        "context_text": row["context_text"],
        "crime_type": row["primary_type"],
        "location_type": row["location_description"],
        "domestic": row["domestic"],
        "arrest": row["arrest"],
        "district": row["district"],
        "year": row["year"]
    })

# ===============================
# STEP 6: Save LLM-3 Dataset
# ===============================
with open("llm3_background_context.json", "w") as f:
    json.dump(llm3_data, f, indent=2)

print("✅ LLM-3 Background & Context Dataset Created")


#Praharshita